## StreamStats API Scraper Automatic

__Description__: Tool to automatically run the [USGS StreamStats tool](https://www.usgs.gov/mission-areas/water-resources/science/streamstats-streamflow-statistics-and-spatial-analysis-tools?qt-science_center_objects=0#qt-science_center_objects) for multiple points within a catchment and return the flow frequency curves and subcatchment boundaries. For additional details, see the [StreamStats Automation Wiki](https://github.com/Dewberry/usgs-tools/wiki/StreamStats-Automation).

__Input__: A shapefile containing the latitude and longitude of points on the stream grid for the specified state (confluence and main stem locations).

__Output__: GeoJSON file containing the delinated catchment boundary and flow frequency data for each point, as well as a CSV file containing the flow frequency data.

*Authors*: sputnam@Dewberry.com & slawler@Dewberry.com

### Load libraries and Python options:

In [1]:
import os
from stream_stats import*
import geopandas as gpd
from geojson import dump

### Specify the state abbreviation and location of the shapefile: 

In [2]:
state='MD' #The state abbreviation

path=r'C:\Users\sputnam\Documents\FEMAFloodPlain\ArcGIS\Data\Converted_Graphics_3.shp' #The location of the shapefile containing the lat/lon of the catchment outlets

##### Load the shapefile:

In [3]:
gdf=gpd.read_file(path) #Read the shapefile as a geopandas dataframe

geom=gdf.geometry #Extract the shapley geometry for the outlets in the shapefile

print(geom.head(2))

0    POINT (-77.04582686669082 38.95039400462227)
1    POINT (-77.05163009280658 38.98614966541195)
Name: geometry, dtype: object


### Run the API tool for each catchment outlet:

In [ ]:
pp_dic = {} #Dictionary to store the outlet flow frequency data dictionaries

watershed_poly_dic= {} #Dictionary to store the catchment polygons (catchment boundaries)

pp_fail=[] #List to store outlet locations whose flow frequency/catchment polygons were not calculated

pp_dic, watershed_poly_dic=snappoint_analysis(geom, state, status=False) #Run the snappoint function for all catchment outlets within the shapefile and for the specified state. Option: set status=False to hide print statements

### Construct a summary table of the flow frequency data for each outlet:

In [ ]:
ffdata=ff_summary(pp_dic) #Run this function to construct the summary table for all outlet locations

ffdata.head()    

###  Save the results:

##### As a CSV:

In [ ]:
cpath=os.path.join(os.getcwd(),'results') #Location to save the results

if os.path.isdir(cpath)==False: #If the desired path does not exist, create it.
    os.mkdir(cpath)    
    
ffdata.to_csv(os.path.join(cpath,'flow_frequency_data.csv')) #Save the results as a csv

##### As a geojson:

In [ ]:
for i in pp_dic.keys():
    watershed_poly_dic[i]['features'][0]['ffcurve']=pp_dic[i]
    
with open('results\catchment_poly.geojson', 'w') as f:
   dump(watershed_poly_dic, f)        

# END